In [ ]:
from pyspark.sql import SparkSession
# Initialize PySpark session
spark = SparkSession.builder \
    .appName("JupyterHub PySpark Example") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate() 

# /!\ Tout se fera à partir de cet object magique `spark`

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, FloatType

In [ ]:
df_beers = spark.read.csv("/datasets/csv/beers.csv", header=True)
df_breweries = spark.read.csv("/datasets/csv/breweries.csv", header=True)

In [ ]:
@F.udf(returnType=FloatType())
def safe_cast_to_float(str_float: str):
    return float(str_float)

df_beers_brewers = (
    df_beers
    .join(df_breweries.withColumnRenamed("name", "brewer_name"), on=df_beers.brewery_id == df_breweries.id)
).cache()

# UC-1

In [ ]:
%%time 
n_beers = df_beers.count()
print(f"Q1: {n_beers} dans la DB")

In [ ]:
%%time
print("Q2")
dd = (df_beers
      .join(df_breweries, on=df_beers.brewery_id == df_breweries.id)
      .groupby("country")
      .count()
      .sort(F.col("count").desc())
      .limit(10)
)
dd.show()

In [ ]:
%%time
# Q3

@F.udf(returnType=FloatType())
def safe_cast_to_float(str_float: str):
    return float(str_float)

df_beers_brewers = (
    df_beers
    .join(df_breweries.withColumnRenamed("name", "brewer_name"), on=df_beers.brewery_id == df_breweries.id)
).cache()

print("Q3")
dd = (df_beers_brewers
      .filter(F.col("country") == F.lit("France"))
      .withColumn("abv_float", safe_cast_to_float(F.col("abv")))
      .sort(F.col("abv_float").desc())
      .select(["name", "abv_float", "country"])
      .limit(10)
)
dd.show()

In [ ]:
%%time
print("Q4")
df_style = spark.read.csv("/datasets/csv/styles.csv", header=True)
target_style_id = df_style.filter(F.lower(F.col("style_name")) == "porter").select(F.col("id").alias("style_id"))
dd = (
    df_beers_brewers
    .join(target_style_id, how="inner", on="style_id")
    .withColumn("abv_float", safe_cast_to_float(F.col("abv")))
    .select(["name", "brewer_name", "abv_float", "country"])
    .groupby("country")
    .agg(F.avg("abv_float").alias("avg_abv"), F.countDistinct("brewer_name").alias("n_brewer_having_porter"))
    .show()
)

In [ ]:
%%time
print("Q5")
dd = (
    df_beers_brewers
    .groupby("country")
    .count()
    .agg(F.median("count"))
)
print("Q5:", dd.first()[0])

# UC-2
Exo difficile

In [ ]:
# output schema : https://stackoverflow.com/a/54771215/10716281

from pyspark.sql.types import *

mapping = {
    "float64": DoubleType,
    "object":StringType,
    "int64":IntegerType,
    "int32":IntegerType,
    "bool": BooleanType,
} # Incomplete - extend with your types.

def createUDFSchemaFromPandas(dfp):
  column_types  = [StructField(key, mapping[str(dfp.dtypes[key])]()) for key in dfp.columns]
  schema = StructType(column_types)
  return schema

In [ ]:
def compute_cascade_model_per_user_query(df: pd.DataFrame) -> pd.DataFrame:
    pos_of_clicked_id = df[df["id_in_serp"] == df["clicked_id"]].iloc[0]["pos_in_serp"]
    df["seen"] = (df["pos_in_serp"] <= pos_of_clicked_id).astype(int)
    df["clicked"] = np.where(df["pos_in_serp"] == pos_of_clicked_id, 1, 0)
    return df

df_processed = compute_cascade_model_per_user_query(df_pref.limit(3).toPandas())
schema = createUDFSchemaFromPandas(df_processed)

In [ ]:
(
    df_pref
    .repartition(16, "query", "user_id")
    .groupby(["query", "user_id"])
    .applyInPandas(compute_cascade_model_per_user_query, schema)
    .filter(F.col("seen") == F.lit(1))
    .groupby(["query", "id_in_serp"])
    .agg(F.sum("seen").alias("n_seen"), F.sum("clicked").alias("n_clicked"))
    .withColumn("clic_proba", F.col("n_clicked") / F.col("n_seen"))
    .select(["query", "id_in_serp", "clic_proba"])
    .show()
)

# UC-3 search from a query

**Observation :** On ne pourra pas aller bien loin en terme de souplesse dans la requête

In [ ]:
df_description = (
    df_beers
    .select(["id", "name", "descript", "brewery_id"])
    .withColumn("beer_text", F.coalesce(F.col("descript"), F.lit("")))
    .withColumnRenamed("name", "beer_name")
    .join(
        (
            df_breweries
            .select(["id", "name", "descript"])
            .withColumn("brewer_text", F.coalesce(F.col("descript"), F.lit("")))
            .withColumnRenamed("name", "brewer_name")
        ), 
        on=df_beers.brewery_id == df_breweries.id
    )
    .withColumn("text", F.concat(F.col("beer_text"), F.lit(". "), F.col("brewer_text")))
    .select(["beer_name", "brewer_name", "text"])
).cache()

In [ ]:
query = "stout"
df_description.filter(F.col("text").contains(query)).toPandas()

# UC-4 vectorize items

In [ ]:
q = """
WITH data AS (
    SELECT 
        beers.id, beers.name, beers.abv, beers.ibu, beers.srm, beers.descript as beer_descr,
        brew.descript as brewer_descript, brew.name as brewery,
        styles.style_name
    FROM beers
    LEFT JOIN breweries as brew on brew.id = beers.brewery_id
    LEFT JOIN styles on styles.id = beers.style_id
), descriptions AS (
    SELECT 
        id,
        CONCAT('the beer ', name, ' from brewery ', brewery, ' (', brewer_descript, ') crafts the beer ', name, ' defined as ', beer_descr, '. Spec of the beer are: ABV=', abv, ', IBU=', ibu, ', SRM=', srm) as to_vectorize
    FROM data
)
SELECT 
    id, to_vectorize
FROM descriptions
WHERE True
    AND id % 12 = 3
;"""
df = pd.read_sql_query(q, con=conn)

In [ ]:
import requests
from typing import List
import numpy as np

class JinaEmbedder:
    
    URL = 'https://api.jina.ai/v1/embeddings'
    EMBEDDING_NAME = "jina-embeddings-v2-base-en"
    bearer_token = 'Bearer jina_85ba1ab9e5ff4017b3d216ebb8734f27xzJ9WyoYBFwqks9lOaNLHryw_Yyz'

    @staticmethod
    def http_json_to_vec(http_json: dict):
        return np.array(
            [
                sentence["embedding"]
                for sentence in http_json["data"]
            ]
        )        

    @classmethod
    def embed(cls, str_to_vectorize: List[str] | str) -> dict:
        if isinstance(str_to_vectorize, str):
            str_to_vectorize = [str_to_vectorize]
        headers = {
            'Content-Type': 'application/json',
            'Authorization': cls.bearer_token
        }
        data = {
            'model': cls.EMBEDDING_NAME,
            'normalized': True,
            'embedding_type': 'float',
            'input': str_to_vectorize
        }
        
        response = requests.post(cls.URL, headers=headers, json=data)

        if response.status_code != 200:
            raise ValueError(f"Error code {response.status_code} on this call")

        return JinaEmbedder.http_json_to_vec(response.json())


In [ ]:
my_embeddings = JinaEmbedder.embed(df.iloc[:, 1].to_list())
np.savez("./embedding-jina-id-3.npz", my_embeddings)

# UC-5 : answer question in corpa
Pas vraiment de possibilité native en SQL